In [1]:
import time
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from IPython.display import display, Latex
import pandas as pd
from bs4 import BeautifulSoup

In [38]:
chrome_options = Options()
chrome_options.add_argument("--headless")  # Enable headless mode
# headless mode should prevent the program from actually launching the ghost browser
# saves time because I only need to visit the site to get the text

driver = webdriver.Chrome(options=chrome_options)

url = 'https://artofproblemsolving.com/wiki/index.php/2023_AMC_12B_Problems/Problem_6'
driver.get(url)





In [30]:
parent = driver.find_element(By.CSS_SELECTOR, '.mw-parser-output')

all_children = parent.find_elements(By.XPATH, './*')

startpoint = parent.find_element(By.ID, 'Problem')

endpoint = parent.find_element(By.ID, 'Solution_1')

problem_elements = []
capture = False



# grab all the elements between problem word element, and answer element

problem = ''
for child in all_children:
    if child.tag_name == 'h2' and not capture:
        capture = True
    elif child.tag_name == 'h2' and capture:
        break
    elif capture:
        curr = ''
        inner_html = child.get_attribute('innerHTML')
        soup = BeautifulSoup(inner_html, 'html.parser')
        for content in soup.descendants:
            if content.name == 'img':
                problem += ' ' + content.get('alt', '')
                curr += ' ' + content.get('alt', '')
            elif isinstance(content, str):
                problem += ' ' + content.strip()
                curr += ' ' + content.strip()
        problem_elements.append(curr)
        
        
answer = problem_elements[-1]
problem = ''.join(problem_elements[:-1])
display(Latex(problem))
display(Latex(answer))
        
        

# python only lets me make this a one liner 
# I think
# answers = parent.find_element(By.XPATH, './/p[2]').find_element(By.CSS_SELECTOR, '.latex').get_attribute('alt')




<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [ ]:
# print(all_children)
# now we need to parse all the elements containing the problems
# problem = ''
# for element in problem_elements:
#     for child in element.find_element(By.XPATH, './node()'):
#         if child.tag_name == '':
#             problem += ' ' + child.text.strip()
#         elif child.tag_name == 'img':
#             problem += ' ' + child.get_attribute('alt')

# if not child.find_elements(By.TAG_NAME, 'img'):
#             print(child.text)
#         else:
#             problem = ''
#             for c in child.find_elements(By.XPATH, './/*'):
#                 if c.tag_name == 'img':
#                     problem += ' ' + c.get_attribute('alt')
#                 else:
#                     problem += ' ' + c.text
#             print(problem)

# repl = ['B', 'C', 'D', 'E']

# for char in repl:
#     answers = answers.replace(char, 'A')

# answers = answers.replace('$', '').replace('\qquad', '')

# answers_arr = answers.split('\\textbf{(A) }')[1:]
# print(answers_arr)

# answer_map = {}

# answer_map['A'] = answers_arr[0]
# answer_map['B'] = answers_arr[1]
# answer_map['C'] = answers_arr[2]
# answer_map['D'] = answers_arr[3]
# answer_map['E'] = answers_arr[4]

# print(answer_map)

In [27]:
columns = ['Contest', 'Year', 'Number', 'Problem', 'Link', 'AnsA', 'AnsB', 'AnsC', 'AnsD', 'AnsE', 'Tags']
contest, year, tags = 'AMC12B', 2023, ['geometry', 'number theory', 'combinatorics'] # just random example tags

df = pd.DataFrame(columns=columns)

for year in range(2002, 2023):
    if year == 2021:
        continue # this one has weird formatting, and the urls are different
    # url = 'https://artofproblemsolving.com/wiki/index.php/' + str(year) + '_AMC_12A_Problems/Problem_'
    for i in range(1, 26):
        if (year == 2003 and i == 16):
            continue # getting errors with 2003 12A #16, not sure why
        number = str(i)
        url = 'https://artofproblemsolving.com/wiki/index.php/' + str(year) + '_AMC_12A_Problems/Problem_' + number
        driver.get(url)
        print(url)
        parent = driver.find_element(By.CSS_SELECTOR, '.mw-parser-output')
        
        all_children = parent.find_elements(By.XPATH, './*')

        startpoint = parent.find_element(By.XPATH,  "//*[contains(@id, 'Problem')]")

        endpoint = parent.find_element(By.XPATH, "//*[contains(@id, 'Solution')]")

        problem_elements = []
        capture = False



        # grab all the elements between problem word element, and answer element
        for child in all_children:
            if child.tag_name == 'h2' and not capture:
                capture = True
            elif child.tag_name == 'h2' and capture:
                break
            elif capture:
                curr = ''
                inner_html = child.get_attribute('innerHTML')
                soup = BeautifulSoup(inner_html, 'html.parser')
                for content in soup.descendants:
                    if content.name == 'img':
                        curr += ' ' + content.get('alt', '')
                    elif isinstance(content, str):
                        curr += ' ' + content.strip()
                problem_elements.append(curr)
                
                
        answers = problem_elements[-1]
        problem = ''.join(problem_elements[:-1])
        # display(Latex(problem))
        # print(answers)


        repl = ['B', 'C', 'D', 'E']

        for char in repl:
            answers = answers.replace(char, 'A')

        answers = answers.replace('$', '').replace('\qquad', '').replace('\\textbf{(A) }', '\\textbf{(A)}').replace('\\textbf{(A)}', '\\text{(A)}')
        answers_arr = answers.split('\\text{(A)}')[1:]

        if not len(answers_arr) == 5:
            print('skipping')
            continue
        if i == 16: print(problem)

        df.loc[len(df)] = ['AMC12A', year, i, problem, url, answers_arr[0], answers_arr[1], answers_arr[2], answers_arr[3], answers_arr[4], tags]
        print(str(year) + " 12A " + str(number))
    
    # url = 'https://artofproblemsolving.com/wiki/index.php/' + str(year) + '_AMC_12B_Problems/Problem_'
    for i in range(1, 26):
        number = str(i)
        url = 'https://artofproblemsolving.com/wiki/index.php/' + str(year) + '_AMC_12B_Problems/Problem_' + number
        driver.get(url)
        print(url)
        parent = driver.find_element(By.CSS_SELECTOR, '.mw-parser-output')
        
        all_children = parent.find_elements(By.XPATH, './*')

        startpoint = parent.find_element(By.XPATH,  "//*[contains(@id, 'Problem')]")

        endpoint = parent.find_element(By.XPATH, "//*[contains(@id, 'Solution')]")

        problem_elements = []
        capture = False



        # grab all the elements between problem word element, and answer element
        for child in all_children:
            if child.tag_name == 'h2' and not capture:
                capture = True
            elif child.tag_name == 'h2' and capture:
                break
            elif capture:
                curr = ''
                inner_html = child.get_attribute('innerHTML')
                soup = BeautifulSoup(inner_html, 'html.parser')
                for content in soup.descendants:
                    if content.name == 'img':
                        curr += ' ' + content.get('alt', '')
                    elif isinstance(content, str):
                        curr += ' ' + content.strip()
                problem_elements.append(curr)
                
                
        answers = problem_elements[-1]
        problem = ''.join(problem_elements[:-1])
        # display(Latex(problem))
        # print(answers)


        repl = ['B', 'C', 'D', 'E']

        for char in repl:
            answers = answers.replace(char, 'A')

        answers = answers.replace('$', '').replace('\qquad', '').replace('\\textbf{(A) }', '\\textbf{(A)}').replace('\\textbf{(A)}', '\\text{(A)}')
        answers_arr = answers.split('\\text{(A)}')[1:]

        if not len(answers_arr) == 5:
            print('skipping')
            continue
        if i == 16: print(problem)

        df.loc[len(df)] = ['AMC12B', year, i, problem, url, answers_arr[0], answers_arr[1], answers_arr[2], answers_arr[3], answers_arr[4], tags]
        print(str(year) + " 12B " + str(number))
    

print(df)
df.to_csv('AMC12B_data', index=False)


https://artofproblemsolving.com/wiki/index.php/2002_AMC_12A_Problems/Problem_1
2002 12A 1
https://artofproblemsolving.com/wiki/index.php/2002_AMC_12A_Problems/Problem_2
2002 12A 2
https://artofproblemsolving.com/wiki/index.php/2002_AMC_12A_Problems/Problem_3
2002 12A 3
https://artofproblemsolving.com/wiki/index.php/2002_AMC_12A_Problems/Problem_4
skipping
https://artofproblemsolving.com/wiki/index.php/2002_AMC_12A_Problems/Problem_5
2002 12A 5
https://artofproblemsolving.com/wiki/index.php/2002_AMC_12A_Problems/Problem_6
2002 12A 6
https://artofproblemsolving.com/wiki/index.php/2002_AMC_12A_Problems/Problem_7
2002 12A 7
https://artofproblemsolving.com/wiki/index.php/2002_AMC_12A_Problems/Problem_8
2002 12A 8
https://artofproblemsolving.com/wiki/index.php/2002_AMC_12A_Problems/Problem_9
2002 12A 9
https://artofproblemsolving.com/wiki/index.php/2002_AMC_12A_Problems/Problem_10
skipping
https://artofproblemsolving.com/wiki/index.php/2002_AMC_12A_Problems/Problem_11
2002 12A 11
https://art

In [40]:
# scrape the solutions now

# url = 'https://artofproblemsolving.com/wiki/index.php/2009_AMC_12A_Problems/Problem_9' # singular URL for now

def get_soup(element):
        inner_html = element.get_attribute('innerHTML')
        soup = BeautifulSoup(inner_html, 'html.parser')
        return soup

def parse_solution(url):
    driver.get(url)
    print(url)
    parent = driver.find_element(By.CSS_SELECTOR, '.mw-parser-output')

    all_children = parent.find_elements(By.XPATH, './*')

    # startpoint = parent.find_element(By.XPATH,  "//*[contains(@id, 'Solution')]")

    # endpoint = parent.find_element(By.XPATH, "//*[contains(@id, 'See_Also')]")

    problem_elements = []
    capture = False

    for child in all_children:
        if child.tag_name == 'h2':
            #     print(child.get_attribute('id'))
            # if child.tag_name == 'h2' and 'Solution' in child.get_attribute('id'):
            #     print('test1')
            #     capture = True
            # elif child.tag_name == 'h2' and 'See_Also' in child.get_attribute('id'):
            #     print('test2')
            #     break
            soup = get_soup(child)
            # I think if the tag is h2 then it should only have 1 descendant
            desc = list(soup.descendants)
            if hasattr(desc[0], 'text'):
                content = desc[0].text.strip()
                if 'Solution' in content:
                    capture = True
                elif content == 'See Also':
                    break
            
        elif capture:       
            curr = ''
            soup = get_soup(child)
            for content in soup.descendants:
                if content.name == 'img':
                    curr += ' ' + content.get('alt', '')
                elif isinstance(content, str):
                    curr += ' ' + content.strip()
            problem_elements.append(curr)

    solution = ''.join(problem_elements)
    print(solution)
    return solution



In [43]:
df = pd.read_csv('AMC12_data.csv')

df['Solution'] = pd.NA

def get_url_a(year, number):
    return 'https://artofproblemsolving.com/wiki/index.php/' + str(year) + '_AMC_12A_Problems/Problem_' + str(number)

def get_url_b(year, number):
    return 'https://artofproblemsolving.com/wiki/index.php/' + str(year) + '_AMC_12B_Problems/Problem_' + str(number)


# for year in range(2002, 2023):
#     if year == 2021:
#         continue
#     for i in range(1, 26):
#         matching_rows = (df['Year'] == year) & (df['Number'] == i) & (df['Contest'] == 'AMC12A')
#         df.loc[matching_rows, 'Solution'] = parse_solution(get_url_a(year, i))

#         matching_rows = (df['Year'] == year) & (df['Number'] == i) & (df['Contest'] == 'AMC12B')
#         df.loc[matching_rows, 'Solution'] = parse_solution(get_url_b(year, i))

def generate_url(contest, year, number):
    return 'https://artofproblemsolving.com/wiki/index.php/' + str(year) + '_AMC_12' + str(contest)[-1] + '_Problems/Problem_' + str(number)
for index, row in df.iterrows():
    # Get the values from the 'Contest', 'Year', and 'Number' columns
    contest = row['Contest']
    year = row['Year']
    number = row['Number']

    # Generate the URL
    url = generate_url(contest, year, number)

    # Parse the solution from the URL
    solution = parse_solution(url)

    # Update the 'Solution' column with the returned value
    df.at[index, 'Solution'] = solution
    
df.to_csv('AMC12_data.csv', index=False)
print(df)
        



https://artofproblemsolving.com/wiki/index.php/2002_AMC_12A_Problems/Problem_1
 We expand to get $2x^2-8x+3x-12+2x^2-12x+3x-18=0$ which is $4x^2-14x-30=0$ after combining like terms. Using the quadratic part of Vieta's Formulas , we find the sum of the roots is $\frac{14}4 = \boxed{\textbf{(A) }7/2}$ . Combine terms to get $(2x+3)\cdot\Big( (x-4)+(x-6) \Big) = (2x+3)(2x-10)=0$ , hence the roots are $-\frac{3}{2}$ and $5$ , thus our answer is $-\frac{3}{2}+5=\boxed{\textbf{(A) } 7/2}$ .   2002 AMC 12A ( Problems • Answer Key • Resources )   Preceded by First Question   Followed by Problem 2    1  •  2  •  3  •  4  •  5  •  6  •  7  •  8  •  9  •  10  •  11  •  12  •  13  •  14  •  15  •  16  •  17  •  18  •  19  •  20  •  21  •  22  •  23  •  24  •  25    All AMC 12 Problems and Solutions    2002 AMC 10A ( Problems • Answer Key • Resources )   Preceded by Problem 9   Followed by Problem 11    1  •  2  •  3  •  4  •  5  •  6  •  7  •  8  •  9  •  10  •  11  •  12  •  13  •  14  •  15  • 

C:\Users\adity\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


 Solution 1 Let $r_1$ and $r_2$ be the radii of circles $A$ and $B$ , respectively. It is well known that in a circle with radius $r$ , a subtended arc opposite an angle of $\theta$ degrees has length $\frac{\theta}{360} \cdot 2\pi r$ . Using that here, the arc of circle A has length $\frac{45}{360}\cdot2\pi{r_1}=\frac{r_1\pi}{4}$ . The arc of circle B has length $\frac{30}{360} \cdot 2\pi{r_2}=\frac{r_2\pi}{6}$ . We know that they are equal, so $\frac{r_1\pi}{4}=\frac{r_2\pi}{6}$ , so we multiply through and simplify to get $\frac{r_1}{r_2}=\frac{2}{3}$ . As all circles are similar to one another, the ratio of the areas is just the square of the ratios of the radii, so our answer is $\boxed{\textbf{(A) } 4/9}$ . Solution 2 The arc of circle $A$ is $\frac{45}{30}=\frac{3}{2}$ that of circle $B$ . The circumference of circle $A$ is $\frac{2}{3}$ that of circle $B$ ( $B$ is the larger circle). The radius of circle $A$ is $\frac{2}{3}$ that of circle $B$ . The area of circle $A$ is ${\lef

In [10]:
import requests
import json
import pandas as pd

# Your OpenAI API key
openai_api_key = 'sk-proj-wNMCoN7x5cQv1F5DzCF0T3BlbkFJ5KQ7B3UUOEsV6NBvCfKb'

# Endpoint URL for embeddings
url = "https://api.openai.com/v1/embeddings"

# Example Pandas Series
texts_series = pd.read_csv('AMC12_data.csv')['Solution']

# Convert the Pandas Series to a list
texts = texts_series.tolist()

# The model you want to use for embeddings, for example, 'text-embedding-ada-002'
model = "text-embedding-ada-002"

# Prepare the headers and data payload
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {openai_api_key}",
}

data = {
    "model": model,
    "input": texts[:1],
}

# Convert the data dictionary to a JSON string
json_data = json.dumps(data)

# Send the POST request to the OpenAI API
response = requests.post(url, headers=headers, data=json_data)
print(response.text[:100])
# Handle the response
if response.status_code == 200:
    response_json = response.json()
    chunk_size = 10  # Print the first 10 items in the list, for example
    print("Subset of JSON Data:")
    for item in response_json[:chunk_size]:
        print(item)
else:
    print("Error:", response.status_code)
    print("Message:", response.json())


{
    "error": {
        "message": "You exceeded your current quota, please check your plan and bil
Error: 429
Message: {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}


In [ ]:
from 